# Water Potability Pipeline Development Notebook

In [1]:
import pyspark
import pyspark.sql.functions as F
import pyspark.sql.types as T
import sys
from pyspark.sql import SparkSession, Window, DataFrame
from pyspark.mllib.stat import Statistics

sys.path.append('/home/jovyan/work')

In [2]:
spark = SparkSession.builder.getOrCreate()